In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-machine-hack/Test.csv
/kaggle/input/fake-news-machine-hack/Train.csv
/kaggle/input/fake-news-machine-hack/sample submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/fake-news-machine-hack/Train.csv")
test = pd.read_csv("/kaggle/input/fake-news-machine-hack/Test.csv")
samp = pd.read_csv("/kaggle/input/fake-news-machine-hack/sample submission.csv")

In [3]:
train.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [4]:
train['Text_Tag'].value_counts()

health-care                                      381
taxes                                            308
immigration                                      253
elections                                        252
education                                        237
                                                ... 
education,state-budget,state-finances,taxes        1
economy,job-accomplishments,message-machine        1
energy,environment,public-safety                   1
government-regulation,marijuana,public-health      1
children,education,legal-issues,sexuality          1
Name: Text_Tag, Length: 3827, dtype: int64

In [5]:
train['Labels'].value_counts()

2    2114
1    1995
3    1962
5    1676
0    1654
4     839
Name: Labels, dtype: int64

In [6]:
train.groupby(['Text_Tag'])['Labels'].value_counts()

Text_Tag                                        Labels
Alcohol                                         5         2
                                                1         1
                                                3         1
Alcohol,animals,children,crime                  5         1
Alcohol,campaign-finance,ethics,public-service  3         1
                                                         ..
workers                                         3         3
                                                1         2
                                                4         2
                                                0         1
                                                5         1
Name: Labels, Length: 5474, dtype: int64

In [7]:
train.groupby(['Labels'])['Text_Tag'].value_counts()

Labels  Text_Tag            
0       health-care             66
        taxes                   50
        immigration             42
        candidates-biography    39
        education               34
                                ..
5       urban                    1
        water                    1
        wealth                   1
        welfare                  1
        workers                  1
Name: Text_Tag, Length: 5474, dtype: int64

In [8]:
test['Text_Tag'].value_counts()

health-care                                           40
elections                                             31
education                                             30
candidates-biography                                  30
economy                                               21
                                                      ..
candidates-biography,crime,criminal-justice,ethics     1
candidates-biography,legal-issues                      1
homeland-security,human-rights,immigration             1
cap-and-trade,climate-change,environment               1
city-budget,labor                                      1
Name: Text_Tag, Length: 732, dtype: int64

In [9]:
!pip install simpletransformers
!pip uninstall -y tokenizers
!pip install transformers

     |████████████████████████████████| 209 kB 403 kB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 884 kB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 3.6 MB/s eta 0:00:011
     |████████████████████████████████| 4.4 MB 8.6 MB/s eta 0:00:01


     |████████████████████████████████| 120 kB 13.0 MB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=3b4024985a424447742f1510d6c51dcdf5ba0729cf8ea26e06abe5e8ce59fc13
  Stored in directory: /root/.cache/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval
  Attempting uninstall: cachetools
    Found existing installation: cachetools 3.1.1
    Uninstalling cachetools-3.1.1:
      Successfully uninstalled cachetools-3.1.1


  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.1.1
    Uninstalling ipykernel-5.1.1:
      Successfully uninstalled ipykernel-5.1.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.45.0
    Uninstalling tqdm-4.45.0:
      Successfully uninstalled tqdm-4.45.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 3.1.0 requires tokenizers==0.8.1.rc2, but you'll have tokenizers 0.7.0 which is incompatible.
jupyterlab-git 0.10.0 requires nbdime<2.0.0,>=1.1.0, but you'll have nbdime 2.0.0 which is incompatible.
all

In [10]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [11]:
import os
import gc
import re
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss

In [12]:
train[train.duplicated()]

,Labels,Text,Text_Tag
1846,1,"Obama says Iran is a 'tiny' country, 'doesn't ...",foreign-policy
2697,1,On repealing the 17th Amendment,"debates,elections,states"
4386,1,On the Trans-Pacific Partnership.,trade
4839,2,During Sherrod Browns past decade as a D.C. po...,"economy,job-accomplishments,jobs"
4940,1,On changing the rules for filibusters on presi...,congressional-rules
6784,1,On support for the Export-Import Bank,trade
8906,5,Says Mitt Romney flip-flopped on abortion.,"abortion,message-machine-2012"
9642,1,On changing the rules for filibusters on presi...,congressional-rules


In [13]:
# print(train.shape)
# train = train.drop_duplicates()
# print(train.shape)

In [14]:
test[test.duplicated()]

,Text,Text_Tag


In [15]:
train = train.rename(columns={'Text' : 'text', 'Labels': 'labels'})
test = test.rename(columns={'Text' : 'text'})

In [16]:
train['text'] = train['text'] + ' ' +  train['Text_Tag'].fillna("").apply(lambda x : ' '.join(x.split(',')))
test['text'] = test['text'] + ' ' +  test['Text_Tag'].fillna("").apply(lambda x : ' '.join(x.split(',')))

In [17]:
train.drop('Text_Tag', axis=1, inplace=True)
test.drop('Text_Tag', axis=1, inplace=True)

In [18]:
print(train['text'].apply(lambda x: len(x.split())).describe())

count    10240.000000
mean        20.178320
std          9.832896
min          3.000000
25%         14.000000
50%         19.000000
75%         25.000000
max        470.000000
Name: text, dtype: float64


In [19]:
print(test['text'].apply(lambda x: len(x.split())).describe([0.8,0.9,0.95,0.99]))

count    1267.000000
mean       20.546172
std        14.948253
min         3.000000
50%        19.000000
80%        26.800000
90%        31.000000
95%        35.000000
99%        43.340000
max       432.000000
Name: text, dtype: float64


In [20]:
train = train[['text','labels']]

In [21]:
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [22]:
model_args = {'train_batch_size': 32, 
              'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'fp16': False,
              'do_lower_case': False,
              'num_train_epochs': 4,
              'max_seq_length': 44,
              'regression': False,
              'manual_seed': 1994,
              "learning_rate": 1e-5,
              #'weight_decay': 0.01,
              "save_eval_checkpoints": False,
              "save_model_every_epoch": False,
              'no_cache':True,
              "silent": True,
              "use_early_stopping": True,
              #"early_stopping_delta": 0.015,
              "early_stopping_metric": "mcc",
              "early_stopping_metric_minimize": False,
              "early_stopping_patience": 3,
              "evaluate_during_training_steps": 1000
              }

In [23]:
def get_model():
    model = ClassificationModel('roberta', 'roberta-large', use_cuda=True, num_labels=6, args=model_args)                            
    return model

In [ ]:
err=[]
y_pred_tot=[]
i=1

fold=StratifiedKFold(n_splits=10, shuffle=True, random_state=1994)

for train_index, test_index in fold.split(train, train['labels']):
    train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
    model = get_model()
    gc.collect()

    model.train_model(train1_trn)
    score, raw_outputs_val, wrong_preds = model.eval_model(train1_val) 
    raw_outputs_val = softmax(raw_outputs_val, axis=1)
    #raw_outputs_val = np.clip(raw_outputs_val,0.05,0.95)
    print('Log_Loss:', log_loss(train1_val['labels'], raw_outputs_val))
    err.append(log_loss(train1_val['labels'], raw_outputs_val))
    predictions, raw_outputs_test = model.predict(test['text'])
    raw_outputs_test = softmax(raw_outputs_test, axis=1) 
    y_pred_tot.append(raw_outputs_test)

In [ ]:
np.mean(err, 0)

In [ ]:
# 1.6813

In [ ]:
y_pred = np.mean(y_pred_tot, 0)

In [ ]:
samp.head()

In [ ]:
samp[['0','1','2','3','4','5']] = y_pred
samp.head()

In [ ]:
train = pd.read_csv("/kaggle/input/fake-news-machine-hack/Train.csv")
test = pd.read_csv("/kaggle/input/fake-news-machine-hack/Test.csv")

In [ ]:
test_ids = []
force_senti = []
for index,row in test.iterrows():
    temp = train[train['Text'] == row['Text']]
    #temp = temp[temp['Text_Tag'] == row['Text_Tag']]
    if temp.shape[0] > 0:
        print(temp.shape)
        force_senti.append(list(set(temp['Labels'].tolist()))[0])
        test_ids.append(index)

In [ ]:
dic = {0:[1,0,0,0,0,0],1:[0,1,0,0,0,0],2:[0,0,1,0,0,0],3:[0,0,0,1,0,0],4:[0,0,0,0,1,0],5:[0,0,0,0,0,1]}
for x,y in zip(test_ids,force_senti):
    #print(samp.iloc[x])
    target = dic[y]
    samp.iloc[x] = target
    #print(samp.iloc[x])

In [ ]:
samp.head()

In [ ]:
samp.to_csv('Sub_v1.0.csv', index=False)